# CMB Angular Power Spectrum Simulation using Validated MFSU Formula
This notebook simulates the Cosmic Microwave Background (CMB) angular power spectrum using the Unified Fractal-Stochastic Model (MFSU) with the final validated formula.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error

# Model constants
ALPHA = 1.0
BETA = 0.1
GAMMA = 0.05
THETA = 0.921


In [ ]:
def hurst_noise(length, hurst=0.7, seed=42):
    np.random.seed(seed)
    white_noise = np.random.normal(size=length)
    cumulative_sum = np.cumsum(white_noise)
    return cumulative_sum / np.max(np.abs(cumulative_sum)) * hurst


In [ ]:
def cl_mfsu(ell, A, theta=THETA, beta=BETA, gamma=GAMMA):
    noise = hurst_noise(len(ell), hurst=theta)
    cl_linear = A * ell ** (-theta)
    cl_nonlinear = -gamma * (cl_linear ** 3)
    return cl_linear + beta * noise + cl_nonlinear


In [ ]:
def cost(params, ell, cl_obs, cl_err):
    A = params[0]
    cl_pred = cl_mfsu(ell, A)
    return mean_squared_error(cl_obs, cl_pred, squared=False)


In [ ]:
data = pd.read_csv('Data/CMB/Planck_TT_2018.csv')
ell = data['ell'].values
cl_obs = data['Cl'].values
cl_err = data['Cl_err'].values


In [ ]:
result = minimize(cost, x0=[1e-10], args=(ell, cl_obs, cl_err), bounds=[(1e-12, 1e-8)])
A_best = result.x[0]
print(f"Best fit A: {A_best:.2e}")


In [ ]:
cl_pred = cl_mfsu(ell, A_best)
rmse_mfsu = mean_squared_error(cl_obs, cl_pred, squared=False)
print(f"RMSE MFSU: {rmse_mfsu:.4e}")


In [ ]:
def cl_lcdm(ell, A):
    return A * ell ** (-2.0)

cl_lcdm_pred = cl_lcdm(ell, A_best)
rmse_lcdm = mean_squared_error(cl_obs, cl_lcdm_pred, squared=False)
print(f"RMSE LCDM: {rmse_lcdm:.4e}")
improvement = (rmse_lcdm - rmse_mfsu) / rmse_lcdm * 100
print(f"Improvement over LCDM: {improvement:.2f}%")


In [ ]:
plt.figure(figsize=(10, 6))
plt.errorbar(ell, cl_obs, yerr=cl_err, fmt='k.', label='Planck 2018 (TT)')
plt.plot(ell, cl_pred, 'r-', label=f'MFSU $\\theta$={THETA}')
plt.plot(ell, cl_lcdm_pred, 'b--', label='LCDM $\\theta$=2.0')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Multipole moment $\\ell$')
plt.ylabel('$C_\\ell$ (arb. units)')
plt.title('CMB Angular Power Spectrum: MFSU (Validated Formula)')
plt.legend()
plt.tight_layout()
plt.savefig('Results/CMB/cmb_mfsu_validated_comparison.png')
plt.show()
